In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
class MyConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size)
        self.bias = np.random.randn(out_channels)
       
    def forward(self, inputs):
        batch_size, in_channels, height, width = inputs.shape
        
        # Рассчитываем размеры выходного изображения
        output_height = (height - self.kernel_size + 2 * self.padding) // self.stride + 1
        output_width = (width - self.kernel_size + 2 * self.padding) // self.stride + 1
        
        padded_inputs = np.pad(inputs, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')
        
        # массив для выходных данных
        outputs = np.zeros((batch_size, self.out_channels, output_height, output_width))
        
        # Проходим по изображениям в батче
        for b in range(batch_size):
            # Проходим по выходным каналам
            for c in range(self.out_channels):
                # Проходим по выходным пикселям
                for h_out in range(output_height):
                    for w_out in range(output_width):
                        h_start = h_out * self.stride
                        w_start = w_out * self.stride
                        h_end = h_start + self.kernel_size
                        w_end = w_start + self.kernel_size
                        
                        outputs[b, c, h_out, w_out] = np.sum(padded_inputs[b, :, h_start:h_end, w_start:w_end] * self.weights[c]) + self.bias[c]
        
        return outputs


In [3]:
# Функция для проведения тестирования
def test_conv2d(inp_in_channels, inp_out_channels, inp_kernel_size, inp_stride, inp_padding):
    np.random.seed(0)
    
    # Создание экземпляра класса
    conv = MyConv2D(in_channels=inp_in_channels, out_channels=inp_out_channels, kernel_size=inp_kernel_size, stride=inp_stride, padding=inp_padding)
    
    # Создание случайных входных данных
    inputs = np.random.randn(1, inp_in_channels, 5, 5)
    
    # Вычисление ожидаемого результата с помощью класса Conv2D из pytorch
    
    inputs_tensor = torch.from_numpy(inputs)
    
    conv_torch = nn.Conv2d(in_channels=inp_in_channels, out_channels=inp_out_channels, kernel_size=inp_kernel_size, stride=inp_stride, padding=inp_padding)
    conv_torch.weight.data = torch.from_numpy(conv.weights)
    conv_torch.bias.data = torch.from_numpy(conv.bias)
    
    expected_outputs_tensor = conv_torch(inputs_tensor).detach().numpy()
    
    # Вычисление выходных данных с помощью нашей реализации
    outputs = conv.forward(inputs)
    
    # Сравнение результатов
    assert np.allclose(outputs, expected_outputs_tensor), "Тест провален!"
    
    print("Тест пройден успешно!")

In [4]:
test_conv2d(3, 2, 3, 1, 1)

Тест пройден успешно!


In [5]:
test_conv2d(6, 32, 2, 1, 3)

Тест пройден успешно!


In [6]:
test_conv2d(4, 16, 4, 2, 4)

Тест пройден успешно!
